In [2]:
import pandas as pd

In [8]:
START_YEAR = 2011
SCHEDULE_FOLDER = '../D-I Cleaned/schedules/'
SCHEDULE_COLS = ['school_id', 'team_id', 'date', 'site', 'opponent', 'opponent_url', 'notes', 'opponent_img_url', 'result', 'contest_url', 'attendance']
ORDERED_COLS = ['contest_url', 'date', 'site', 'school_name', 'wl', 'gs', 'ga', 'opponent', 'school_id', 'opponent_school_id', 'attendance', 'notes']
RENAMED_COLS = ['contest_id','date', 'site', 'school', 'result', 'gs', 'ga', 'opponent_school', 'school_id', 'opponent_school_id', 'attendance', 'notes']
years = [str(year-1) + '-' + str(year % 100).zfill(2) for year in range(START_YEAR, 2024)]
schools = pd.read_csv('../general_data/schools.csv', skiprows=1, header=None)
schools.columns = ['school_id', 'school_name']

In [11]:
def clean_schedule(s, pre2017):
    s.columns = SCHEDULE_COLS
    s['site'] = s.site.where(s.site != '@', 'away')
    s['site'] = s.site.where((s.site == 'away') | s.notes.isna() | (s.notes.str[0] != '@'), 'neutral')
    s['site'] = s.site.fillna('home')
    s = s.merge(schools, on='school_id')
    non_d1_games = s.contest_url.value_counts()[s.contest_url.value_counts() == 1].index
    s = s.loc[~s.contest_url.isin(non_d1_games)]
    s = s.loc[(s.result != "Canceled") & (s.result.notna()) & (s.result != 'Ppd')]
    s.loc[:,'result'] = s.result.str.replace('-', ' ')
    game_result_list = s.result.str.split().str
    # print(game_result_list)
    try:
        s['wl'] = s.result.str.split().str[0]
        s['gs'] = s.result.str.split().str[1].astype(int)
        s['ga'] = s.result.str.split().str[2]
    except Exception as e:
        print(e)
        return None
    s = s.merge(s[['school_id', 'contest_url']].add_prefix('opponent_'), left_on='contest_url', right_on='opponent_contest_url', how='left')
    s = s.loc[s.school_id != s.opponent_school_id].drop(columns='opponent_contest_url')
    # s['opponent_school_id'] = s.opponent_url.str.split('/').str[2] if pre2017 else None
    s = s[ORDERED_COLS]
    s['contest_url'] = s.contest_url.str.split('/').str[2]
    s.columns = RENAMED_COLS
    s.loc[:, 'date'] = pd.to_datetime(s.date)
    return s

In [12]:
schedules = [clean_schedule(s, True) for s in [pd.read_csv(SCHEDULE_FOLDER + year + '.csv') for year in years]]

In [17]:
pd.concat(schedules).to_csv('games_2011_to_2023.csv', index=False)

# Finding errors in home/away/neutral

In [ ]:
g = schedules[2]
set(g.loc[g.site == 'away'].contest_id) - set(g.loc[g.site == 'home'].contest_id)
schedules[2].loc[schedules[2].contest_id.isin(schedules[2][['contest_id','site']].loc[schedules[2].site == 'away'].contest_id)].site.value_counts()
st = pd.read_csv('D-I Cleaned/schedules/2023-24.csv', header=None)
st[9] = st[9].str.split('/').str[2]
cs = schedules[-1][['contest_id','site']]
cs.loc[cs.site == 'neutral'].value_counts().sort_values()